# 🔹 3. 训练与优化

## 3.1、优化器 (torch.optim)

optim.SGD

optim.Adam

optim.AdamW

optim.RMSprop

optim.Adagrad

## 3.2、学习率调度器 (torch.optim.lr_scheduler)

StepLR

ExponentialLR 

CosineAnnealingLR

ReduceLROnPlateau

---

# 训练
## 搭建训练循环
- 核心思想：
    - 1.前向传播 (forward)
    - 2.计算损失 (loss)
    - 3.反向传播 (backward)
    - 4.参数更新 (optimizer.step)

In [ ]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0  # 设置初始参数

    for data, target in loader:
        data, target = data.to(device), target.to(device) # 传递到 GPU上运算

        optimizer.zero_grad()         # 梯度清零
        output = model(data)          # 前向传播
        loss = criterion(output, target)  # 计算损失
        loss.backward()               # 反向传播
        optimizer.step()              # 更新参数

        total_loss += loss.item()
        total_correct += (output.argmax(1) == target).sum().item()

    acc = total_correct / len(loader.dataset)
    return total_loss / len(loader), acc


## 通用的模型训练循环

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import os

def train_model(model, train_loader, val_loader, loss_fn, 
                optimizer, device, num_epochs=10, scheduler=None, 
                save_dir='path', early_stopping_patience=None):
    """
    通用的模型训练循环
    
    参数:
        model: 要训练的模型
        train_loader: 训练数据加载器
        val_loader: 验证数据加载器
        loss_fn: 损失函数
        optimizer: 优化器
        device: 设备 (cpu/cuda)
        num_epochs: 训练轮数
        scheduler: 学习率调度器 (可选)
        save_dir: 检查点保存目录
        early_stopping_patience: 早停耐心值 (可选)
    
    返回:
        训练历史记录，包含损失和准确率
    """
    # 创建保存目录
    os.makedirs(save_dir, exist_ok=True)
    
    # 初始化历史记录
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    # 早停相关变量
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    # 训练循环
    for epoch in range(num_epochs):
        # 训练阶段
        model.train()
        train_loss, train_correct = 0, 0 # 初始参数
        train_batches = 0
        
        # 使用tqdm添加进度条
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for data, target in train_pbar:
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_correct += (output.argmax(1) == target).sum().item()
            train_batches += 1
            
            # 更新进度条
            train_pbar.set_postfix({
                'Loss': f'{train_loss/train_batches:.4f}',
                'Acc': f'{100*train_correct/(train_batches * train_loader.batch_size):.2f}%'
            })
        
        # 计算训练指标
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_correct / len(train_loader.dataset)
        
        # 验证阶段
        avg_val_loss, avg_val_acc = validate(model, val_loader, loss_fn, device)
        
        # 更新学习率调度器
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(avg_val_loss)
            else:
                scheduler.step()
        
        # 记录历史
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(avg_val_acc)
        
        # 打印epoch结果
        print(f'Epoch {epoch+1}/{num_epochs}: '
              f'Train Loss: {avg_train_loss:.4f}, Train Acc: {100*avg_train_acc:.2f}% | '
              f'Val Loss: {avg_val_loss:.4f}, Val Acc: {100*avg_val_acc:.2f}%')
        
        # 保存最佳模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_val_loss,
                'acc': avg_val_acc
            }, os.path.join(save_dir, 'best_model.pth'))
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        # 早停检查
        if early_stopping_patience and epochs_no_improve >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch+1} epochs!')
            break
    
    # 保存最终模型
    torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, os.path.join(save_dir, 'final_model.pth'))
    
    return history

def validate(model, loader, loss_fn, device):
    """验证函数"""
    model.eval()
    total_loss, total_correct = 0, 0
    
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)
            
            total_loss += loss.item()
            total_correct += (output.argmax(1) == target).sum().item()
    
    avg_loss = total_loss / len(loader)
    avg_acc = total_correct / len(loader.dataset)
    
    return avg_loss, avg_acc

# 使用示例
if __name__ == "__main__":
    # 假设已有以下组件
    # model = YourModel()
    # train_loader = DataLoader(...)
    # val_loader = DataLoader(...)
    # loss_fn = nn.CrossEntropyLoss()
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 训练模型
    # history = train_model(
    #     model=model,
    #     train_loader=train_loader,
    #     val_loader=val_loader,
    #     loss_fn=loss_fn,
    #     optimizer=optimizer,
    #     device=device,
    #     num_epochs=50,
    #     scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1),
    #     save_dir='model_checkpoints',
    #     early_stopping_patience=5
    # )
    pass

# 3.1、核心概念：torch.optim 是什么？
`torch.optim `是一个包含了多种优化算法 (Optimization Algorithms) 的包。

在神经网络的训练过程中，我们通过以下步骤进行：
- `1、前向传播 (Forward Pass)`：将数据输入模型，得到预测结果。

- `2、计算损失 (Compute Loss)`：将预测结果与真实标签进行比较，计算出损失值（即模型犯了多大的错）。

- `3、反向传播 (Backward Pass)`：根据损失值，计算出模型中每个参数（权重和偏置）的梯度 (`gradient`)。梯度指明了参数应该朝哪个方向调整才能使损失变小。

- `4、更新参数 (Update Parameters)`：这就是 `torch.optim` 发挥作用的地方。优化器根据计算出的梯度，使用特定的优化算法（如 `SGD`, `Adam`）来更新模型的每一个参数，从而使损失函数的值逐渐减小。

简单来说，如果说损失函数是“地图”，梯度是“方向盘”，那么 优化器就是“引擎”，它决定了我们如何驱动模型参数这辆“车”，一步步驶向地图上的最低点（损失最小化）。

---

## torch.optim 的核心使用三步法
在任何 PyTorch 训练循环中，使用优化器的步骤都遵循一个固定的“三步法”模式。

### 1. 初始化优化器 (Initialize the Optimizer)
在训练开始之前，你需要创建一个优化器实例。创建时，你需要告诉它两件事：

- 1 `要优化的参数是哪些？` 通常是模型的所有参数，通过 `model.parameters()` 来获取。

- 2 `使用什么超参数？` 最重要的超参数是学习率 (learning rate, `lr`)，它控制了每次参数更新的步长。


In [ ]:
import torch.optim as optim
from torch import nn

# 假设 model 是一个已经定义好的 nn.Module 实例
model = nn.Linear(10, 2) 

# 创建一个 SGD 优化器
# 第一个参数：告诉优化器需要更新的参数
# lr：设置学习率
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### 2. 梯度清零 (optimizer.zero_grad())
在计算新一轮梯度之前，必须手动将上一步的梯度清零。

- `为什么需要清零？` 因为 PyTorch 的设计是，每次调用 `loss.backward()` 时，计算出的梯度会累加到之前的梯度上。如果不清零，梯度会越积越大，导致错误的更新方向。

- `调用时机`：通常在每个训练迭代（`batch`）的开始。

---

### 3. 更新参数 (optimizer.step())
当梯度计算完毕（即调用 `loss.backward()` 之后），调用 `optimizer.step()`。

- `作用`：优化器会遍历所有它管理的参数，并根据存储在这些参数` .grad` 属性中的梯度，使用其内部定义的优化算法来更新参数的值。
#### 这三步在训练循环中的位置如下：

In [ ]:
# 假设 train_loader 是我们的数据加载器
for inputs, labels in train_loader:
    # 1. 梯度清零
    optimizer.zero_grad()
    
    # 2. 前向传播
    outputs = model(inputs)
    
    # 3. 计算损失
    loss = criterion(outputs, labels)
    
    # 4. 反向传播，计算梯度
    loss.backward()
    
    # 5. 更新参数
    optimizer.step()

---
## 常见优化器详解
torch.optim 提供了多种优化器，我们介绍最常用也是最重要的几个。

---
### 1. optim.SGD (随机梯度下降)
最基础、最经典的优化器。

- `核心思想`：参数沿着梯度的反方向前进一步。步子的大小由学习率 `lr` 控制。

公式简化版: weight=weight−learning_rate
timesgradient

- 常用参数:

  - **`params`**: 模型参数。

  - **`lr`**: 学习率。

  - **`momentum`**: 动量。引入动量可以帮助加速梯度下降，并冲出局部最优。它模拟了物理世界中物体的惯性，使得更新方向不仅取决于当前梯度，也受之前更新方向的影响。`通常设为 0.9`。

  - **`weight_decay`**: 权重衰减（`L2` 正则化）。用于防止过拟合，通过给损失函数增加一个惩罚项来限制模型权重的大小。

---
### 2. optim.Adam (Adaptive Moment Estimation)
目前最流行、最常用的优化器之一，通常作为各种任务的默认首选。

- `核心思想`：它是一种自适应学习率算法。它不满足于所有参数共享同一个学习率，而是为每个参数计算自适应的学习率。

- `优点`：结合了 `Momentum` 和 `RMSprop` 两种算法的思想，能够快速收敛，并且对超参数的选择不如 `SGD` 那么敏感。

- 常用参数:

  - **`lr`**: 初始学习率。

  - **`betas`**: 用于计算梯度的一阶和二阶矩估计的系数，通常保持默认值 `(0.9, 0.999)`。

  - **`eps`**: 为了增加数值稳定性而加到分母的一个小项，通常保持默认值 `1e-8`。

  - **`weight_decay`**: 权重衰减。

---
### 3. optim.AdamW
`AdamW` 是对 `Adam` 中权重衰减实现方式的改进版本。在 `Adam` 中，权重衰减与梯度更新耦合在一起，效果有时并不理想。`AdamW` 将其解耦，在许多任务中（尤其是 `NLP` 领域的 `Transformer` 模型）表现得比 `Adam` 更好，正在成为新的标准。

---
### 4. optim.Adagrad (Adaptive Gradient Algorithm)
Adagrad 是最早期的自适应学习率算法之一。

- 名称解读 \
Adaptive Gradient：自适应梯度。

- 核心思想 \
`Adagrad` 的核心在于，它会记录下到目前为止，每个参数所有梯度值的平方和。在更新参数时，学习率会除以这个累积值的平方根。

- 直观理解：

  - 如果一个参数的梯度一直很大（经常被更新），那么它的梯度平方和就会很大，导致分母很大，从而使得它的有效学习率变小。

  - 如果一个参数的梯度一直很小或很稀疏（偶尔被更新），那么它的梯度平方和就会很小，导致分母很小，从而使得它的有效学习率变大。

`优点`
   - 自适应学习率：能够自动为不同参数调整学习率，无需手动调整。

   - 特别适合稀疏数据：在处理像词嵌入 (word embeddings) 或推荐系统这类稀疏特征时非常有效。因为稀疏特征对应的参数不常被更新，Adagrad 会给予它们较高的学习率。

`致命弱点` 
   - `Adagrad` 的主要问题在于其分母中的梯度平方和是单调递增的。随着训练的进行，这个累积值会越来越大，永不减小。

   - `后果`：训练后期，几乎所有参数的有效学习率都会变得无限接近于 0，导致模型过早地停止学习，无法达到最优解。这就像一辆车，开着开着油门就自动踩到底，再也加不上速了。

`适用场景`
 - 主要用于处理稀疏数据，如自然语言处理中的词嵌入任务。

 - 在现代的深度学习任务中（如计算机视觉），由于其学习率会过早衰减的问题，现在已经较少使用。


---
### optim.RMSprop (Root Mean Square Propagation)
为了解决 `Adagrad` 学习率急剧下降的问题，Geoff Hinton 提出了 `RMSprop` 算法。它是 `Adam` 优化器的重要组成部分。

`名称解读` \
Root Mean Square Propagation：均方根传播。

`核心思想` \
`RMSprop` 对 `Adagrad` 做了一个简单而巧妙的修改：它不再累加所有的历史梯度平方，而是计算一个梯度的平方的指数移动平均值 (Exponentially Moving Average)。

`直观理解`：

   - 指数移动平均值会给予最近的梯度更高的权重，而逐渐“忘记”久远的梯度。

   - 这使得分母不会无限地、单调地增长。它会根据最近的梯度情况进行动态调整。如果最近梯度大，分母就大，学习率就小；如果最近梯度小，分母就小，学习率就大。

这样一来，RMSprop 解决了 Adagrad 学习率过早消失的问题，让训练可以持续进行。

`优点` \
   - 解决了 `Adagrad` 的弊端：通过使用指数移动平均，避免了学习率过早衰减，使得算法在非凸优化问题上表现更好。

   - 收敛速度快：作为一种自适应学习率算法，通常比 `SGD` 收敛更快。

   - 超参数较少：通常只需要调整学习率 lr 和 alpha (平滑常数)。

`缺点`  \
   - 仍然需要设置一个全局的学习率 lr。

   - 在某些情况下，Adam 或 AdamW 可能会提供更稳定和更好的性能。

`适用场景`  \
 - 一个非常通用和强大的优化器，尤其在循环神经网络 (RNNs) 相关的任务上表现出色。

 - 当你发现 Adam 在你的任务上效果不佳时，RMSprop 是一个非常值得尝试的替代方案。

`进化之路`：你可以这样理解优化器的发展： \
`SGD` -> `SGD with Momentum` -> `Adagrad` (引入自适应思想) -> `RMSprop` (修复 Adagrad 缺陷) -> `Adam` (结合 RMSprop 和 Momentum) -> `AdamW` (改进 Adam 的权重衰减)。

---
## 优化器对比表

| 优化器 (Optimizer) | 核心思想 (Core Idea) | 学习率特点 (Learning Rate) | 优点 (Pros) | 缺点 (Cons) | 适用场景 (Best Use Cases) |
|--------------------|----------------------|----------------------------|------------|------------|----------------------------|
| SGD | 沿着梯度反方向更新参数。可加入动量（Momentum）来增加惯性。 | 全局固定：所有参数共享同一个学习率。 | 简单可靠，经过充分调优后可能找到泛化能力更好的解。 | 对学习率敏感，收敛慢，容易陷入局部最优或鞍点。 | 作为研究基线；当你有足够的时间和计算资源进行精细调参时。 |
| Adagrad | 为每个参数累加所有历史梯度的平方，并用它来调整学习率。 | 自适应：每个参数有独立学习率。梯度大的参数，学习率衰减快。 | 对稀疏数据非常有效（如NLP词嵌入）。 | 学习率单调递减，训练后期会变得极小，导致过早停止学习。 | 主要用于稀疏数据任务，在深度学习中已较少使用。 |
| RMSprop | 使用梯度平方的指数移动平均来调整学习率。 | 自适应：每个参数有独立学习率，但解决了学习率消失问题。 | 解决了 Adagrad 的弊端，收敛速度快。 | 仍然需要手动设置学习率。 | 在循环神经网络（RNN）上表现出色；可作为 Adam 的替代方案。 |
| `Adam` | 结合了 Momentum 和 RMSprop。同时使用梯度的一阶矩（动量）和二阶矩（自适应学习率）。 | 自适应：每个参数有独立学习率，且更新带有惯性。 | 收敛速度极快，对超参数不敏感，`几乎适用于所有任务`。 | 权重衰减的实现方式存在问题；可能收敛到泛化能力较差的“锐利”最小值。 | 曾经的默认首选。适合快速原型设计和大多数深度学习任务。 |
| `AdamW` | 修复了 Adam 的权重衰减问题，将其与梯度更新解耦。 | 自适应：与 Adam 相同，但正则化效果更好。 | 拥有 Adam 的所有优点，同时通过修正权重衰减，通常能获得更好的泛化性能和更低的训练损失。 | `几乎没有明显缺点`。 | 当前推荐的`默认首选`。尤其在 Transformer、BERT 等大型模型上表现优异。 |

---

# 3.2、学习率调整策略 (torch.optim.lr_scheduler)
torch.optim.lr_scheduler。

- `核心概念`：为什么需要学习率调整策略？ \
在之前的 `torch.optim` 讲解中，我们为优化器设置了一个固定的学习率（learning rate, `lr`）。然而，在整个训练过程中使用一个固定的学习率并非最优选择。

- 一个理想的学习率策略应该是这样：

  - 训练初期: 模型参数是随机初始化的，离最优解很远。此时，我们希望使用一个较大的学习率，让模型能够快速地向最优解的方向收敛，就像下山初期大步流星。

  - 训练中期: 当模型接近最优解时，一个大的学习率可能会导致模型在最优解附近“来回震荡”，无法精确收敛。

  - 训练后期: 我们需要一个较小的学习率，让模型能够“小心翼翼”地在最优解的“山谷”中进行微调，从而找到那个最低点。

`torch.optim.lr_scheduler` 就是为了实现这种动态调整学习率的需求而设计的。它提供了多种预设的策略，可以根据训练的进度（如 epoch 数）或其他指标（如验证集损失）来自动调整优化器中的学习率。

---
### 使用方法:

使用任何 lr_scheduler 都遵循一个简单的三步流程：

- 1 、`创建优化器 (Optimizer)`：首先，像往常一样创建一个优化器。

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

- 2 、`创建调度器 (Scheduler)`：然后，用创建好的优化器来实例化一个调度器。

from torch.optim.lr_scheduler import StepLR  

scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

- 3 、`在训练循环中更新调度器 (scheduler.step())`: 在训练循环的适当位置调用 `scheduler.step()` 来更新学习率。

- `step()` 的调用时机是关键：

  - 大多数调度器: 都是基于 epoch 进行更新的，所以 `scheduler.step()` 应该在每个 `epoch` 的训练循环之后调用。

  - 少数调度器: 如 `CosineAnnealingLR`, `OneCycleLR` 等，可以基于 `batch`/`iteration` 进行更新，此时 `scheduler.step()` 应该在每个 `batch` 的训练步骤之后调用。

  - 特殊调度器: `ReduceLROnPlateau` 是基于验证指标更新的，所以它的 `step()` 需要传入该指标的值，例如 scheduler.step(validation_loss)。


---
### 常用 Scheduler:
下面我们介绍几种最常用、最有代表性的调度器。

## 1. StepLR (阶梯式下降)
- 一句话解释: 每隔 `step_size` 个 epoch，就将当前学习率乘以一个 `gamma` 因子。

- `核心参数`:

   - `optimizer`: 关联的优化器。

   - `step_size (int)`: 更新学习率的间隔 `epoch` 数。

   - `gamma (float)`: 学习率衰减的乘法因子（例如，0.1 代表衰减为原来的1/10）。

- 学习率曲线: 呈阶梯状下降。

- 适用场景: 简单有效，适合作为入门和基线。

In [ ]:
# 每 10 个 epoch，学习率变为原来的 0.5 倍

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## 2. MultiStepLR (多阶梯式下降)
- 一句话解释: 在预先设定好的一系列 `milestones` (里程碑) `epoch` 处，将学习率乘以 `gamma`。

- `核心参数`:

    - `milestones (list of ints)`: 一个包含 epoch 索引的列表，在这些 epoch 处进行学习率衰减。

    - `gamma (float)`: 衰减因子。

- 学习率曲线: 不规则的阶梯状，更具灵活性。

- 适用场景: 许多经典论文（如 ResNet）中使用的标准策略，当你对训练过程有比较明确的预期时（例如，你知道模型大概在第60和第90个epoch会遇到瓶颈）。

In [ ]:
# 在第 60, 90, 120 个 epoch 时，学习率分别衰减为原来的 0.1 倍
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 90, 120], gamma=0.1)

## 3. CosineAnnealingLR (余弦退火)
- 一句话解释: 学习率在一个周期（`T_max` 个 epoch）内，按照余弦函数的形式从初始值平滑地下降到最小值 `eta_min`。

- `核心参数`:

    - `T_max (int)`: 一个学习率周期的长度（以 `epoch`(周期) 或 `iteration`(迭代次数) 为单位）。

    - `eta_min (float)`: 学习率的下限，默认为 0。

- 学习率曲线: 平滑的余弦半周期曲线。

`特点`: 这是一种非常流行且高效的策略。它在前期保持较高的学习率，然后缓慢下降，在周期末期学习率变得非常小，有助于模型找到更优的解。还可以配合`“热重启”`（Warm Restarts）使用，让模型有机会跳出局部最优。

`适用场景`: 当前各种深度学习任务中的 `SOTA` (State-of-the-art) 选择，尤其在大型模型训练中表现优异。

In [ ]:
# 在 100 个 epoch 内，学习率从初始值退火到 0

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)

## 4. ReduceLROnPlateau (在平台期降低学习率)
- 一句话解释: 监控一个指定的指标（如验证集损失），如果该指标在 `patience` 个 `epoch`abs 内没有改善，就降低学习率。

- `核心参数`:

    - `mode (str)`: '`mi`' 或 '`max`'。监控指标是越小越好 ('min'，如损失) 还是越大越好 ('max'，如准确率)。

    - `factor (float)`: 学习率衰减因子 (`new_lr` = `lr` * `factor`)。

    - `patience (int)`: 容忍多少个 `epoch` 指标没有改善。

    - `verbose (bool)`: 如果为 `True`，每次更新时会打印一条信息。

`特点`: 这是一种`“响应式”`策略，而不是预设的。它根据模型的实际表现来调整学习率，非常智能。

- `step()` 调用方式: 特别注意，它需要在验证阶段后调用，并传入监控的指标值：`scheduler.step(validation_loss)`。

In [ ]:
# 监控验证集损失，如果连续 5 个 epoch 损失没有下降，则学习率变为原来的 0.2 倍

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5, verbose=True)

## 完整代码演示

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR

# 1. 虚拟的模型、数据和优化器
model = nn.Linear(10, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 2. 创建调度器 (选择一个)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1) # 方案一：阶梯下降
scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=0.001) # 方案二：余弦退火

num_epochs = 50

print("开始训练...")
for epoch in range(num_epochs):
    # ----- 模拟训练循环 -----
    for i in range(10): # 假设有 10 个 batch
        # ... your training steps ...
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        pass # 此处省略训练细节
    # -----------------------
    
    # 在每个 epoch 结束后，更新学习率
    scheduler.step()
    
    # 打印当前学习率以观察变化
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{num_epochs}, 当前学习率: {current_lr:.6f}")

### 如何选择？
- `简单基线`: `StepLR` 或 `MultiStepLR` 是非常好的起点，稳定可靠。

- `追求更高性能`: `CosineAnnealingLR` 是目前冲击 `SOTA` 性能的首选策略之一。

- `需要自适应调整`: 当你不确定训练多少个 epoch 会遇到瓶颈时，`ReduceLROnPlateau` 是一个非常智能和方便的选择，它让模型自己决定何时降低学习率。

---

## 总结
- `torch.optim` 是 PyTorch 的优化核心，负责根据梯度更新模型参数。

- 掌握 `optimizer.zero_grad()` -> `loss.backward()` -> `optimizer.step()` 的`“三步曲”`是编写任何 PyTorch 训练代码的基础。

- `SGD` 是基础，`Adam` 和 `AdamW` 是当前最常用、效果最好的优化器之一。

配合使用 `torch.optim.lr_scheduler` 动态调整学习率是提升模型最终性能的标准实践。